<a href="https://colab.research.google.com/github/kunalsonalkar/training-llms/blob/main/fineTuning_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rasbt/LLMs-from-scratch.git

Cloning into 'LLMs-from-scratch'...
remote: Enumerating objects: 5555, done.
remote: Total 5555 (delta 0), reused 0 (delta 0), pack-reused 5555 (from 1)
Receiving objects: 100% (5555/5555), 12.92 MiB | 9.54 MiB/s, done.
Resolving deltas: 100% (3479/3479), done.


In [2]:
import pandas as pd

try:
    labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='latin1',usecols=[0,1])
except UnicodeDecodeError:
    labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='cp1252', usecols=[0,1])
# If none of the encoding above work, you may also try:
# labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='utf-16')
# labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='iso-8859-1')

In [3]:
import pandas as pd

# Your category-to-integer mapping
category_map = {
    'baby_accessories_a28': 0,
    'bags_a17': 1,
    'belts__braces_a26': 2,
    'bottoms_a3': 3,
    'dresses_a1': 4,
    'eyewear_a19': 5,
    'food_a38': 6,
    'fragrance_a31': 7,
    'gloves_mittens_a21': 8,
    'hair_accessories_a20': 9,
    'hair_care_a33': 10,
    'headwear_a18': 11,
    'home_a35': 12,
    'hosiery_a16': 13,
    'jacket_sportcoat_mg12': 14,
    'jewelry_a14': 15,
    'jewelry_care_a15': 16,
    'jumpsuits_coveralls_a4': 17,
    'makeup_a30': 18,
    'neckwear_a27': 19,
    'outerwear_a11': 20,
    'personal_care_accessories_a34': 21,
    'scarves_wraps_ponchos_a22': 22,
    'shoe_care_a13': 23,
    'shoes_a12': 24,
    'skin_body_treatment_a32': 25,
    'sleepwear_a9': 26,
    'small_leather_goods_a23': 27,
    'stationery_giftwrap_a36': 28,
    'suits_sets_wardrobers_mg5': 29,
    'swimwear_a8': 30,
    'toys_games_a37': 31,
    'tops_a2': 32,
    'underwear_lingerie_a7': 33
}


# Map categories to integers
labels['category_int'] = labels['product_type_1_label'].map(category_map)
print(labels)


              search_term     product_type_1_label  category_int
0             womens uggs                shoes_a12            24
1       charlotte tilbury               makeup_a30            18
2     womens birkenstocks                shoes_a12            24
3                    uggs                shoes_a12            24
4            womens nikes                shoes_a12            24
...                   ...                      ...           ...
3684        girls pajamas             sleepwear_a9            26
3685          hand towels                 home_a35            12
3686            hand wash  skin_body_treatment_a32            25
3687       l'agence jeans               bottoms_a3             3
3688             lipgloss               makeup_a30            18

[3689 rows x 3 columns]


In [4]:
def random_split(df, train_frac=0.7, validation_frac=0.2):
  df = df.sample(frac=1, random_state=42).reset_index(drop=True)
  train_end = int(len(df) * train_frac)
  validation_end = train_end + int(len(df) * validation_frac)
  train_df = df[:train_end]
  validation_df = df[train_end:validation_end]
  test_df = df[validation_end:]
  return train_df, validation_df, test_df

In [5]:
train_df, validation_df, test_df = random_split(labels, train_frac=0.7, validation_frac=0.1)

In [6]:
train_df.to_csv('train.csv', index=None)
validation_df.to_csv('validation.csv', index=None)
test_df.to_csv('test.csv', index=None)

In [7]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.0 MB/s eta 0:00:00


In [8]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.decode([50256]))

<|endoftext|>


In [9]:
import torch
from torch.utils.data import Dataset

class queryIntent(Dataset):
  def __init__(self, csv_file, tokenizer, max_length = None, pad_token_id=50256):
      self.data = pd.read_csv(csv_file)
      self.encoded_texts = [
          tokenizer.encode(text) for text in self.data["search_term"]
      ]
      if max_length is None:
        self.max_length = self._longest_encoded_length()
      else:
        self.max_length = max_length
        self.encoded_texts = [
            encoded_text[:self.max_length] for encoded_text in self.encoded_texts
        ]

      self.encoded_texts = [
          encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
          for encoded_text in self.encoded_texts
      ]

  def __getitem__(self, index):
      encoded = self.encoded_texts[index]
      label = self.data.iloc[index]["category_int"]
      return (
          torch.tensor(encoded, dtype=torch.long),
          torch.tensor(label, dtype=torch.long)
      )

  def __len__(self):
      return len(self.data)

  def _longest_encoded_length(self):
      max_length = 0
      for encoded_text in self.encoded_texts:
          encoded_length = len(encoded_text)
          if encoded_length > max_length:
              max_length = encoded_length
      return max_length

In [10]:
train_dataset = queryIntent("train.csv", max_length=None, tokenizer=tokenizer)
validation_dataset = queryIntent("validation.csv", max_length = None, tokenizer=tokenizer)
test_dataset = queryIntent("test.csv", max_length = None, tokenizer=tokenizer)

In [11]:
from torch.utils.data import DataLoader
num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True
)

validation_loader = DataLoader(
    dataset=validation_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False
)

In [12]:
len(train_loader)

322

In [13]:
len(test_loader)

93

In [14]:
len(validation_loader)

46

In [15]:
len(train_dataset)

2582

In [16]:
for input_batch, label_batch in train_loader:
    print(input_batch.shape)
    print(label_batch.shape)
    break

torch.Size([8, 11])
torch.Size([8])


**Initialize the model and Pretrained Weights**

In [17]:
CHOOSE_MODEL = "gpt2-small (124M)"

BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True,
    "emb_dim": 768,
    "n_layers": 12,
    "n_heads": 12,
    "n_embd": 768
}

model_configs = {
    "gpt2-small (124M)": {
        "n_layer": 12,
        "n_head": 12,
        "n_embd": 768
    },
    "gpt2-medium (355M)": {
        "n_layer": 24,
        "n_head": 16,
        "n_embd": 1024
    },
    "gpt2-large (774M)": {
        "n_layer": 36,
        "n_head": 20,
        "n_embd": 1280
    },
    "gpt2-xl (1.5B)": {
        "n_layer": 48,
        "n_head": 25,
        "n_embd": 1600
    }
}

In [18]:
%cd /content/LLMs-from-scratch/ch06/01_main-chapter-code

/content/LLMs-from-scratch/ch06/01_main-chapter-code


In [19]:
import gpt_download
from gpt_download import download_and_load_gpt2

In [20]:
from previous_chapters import GPTModel, load_weights_into_gpt

In [21]:
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir = "gpt2")

model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 102kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 2.48MiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 140kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [00:42<00:00, 11.7MiB/s]
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 4.37MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 1.37MiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 1.45MiB/s]


In [22]:
from previous_chapters import (
    generate_text_simple,
    text_to_token_ids,
    token_ids_to_text
)

text1 = "Every effort moves you"

token_ids = generate_text_simple(
    model = model,
    idx = text_to_token_ids(text1, tokenizer),
    max_new_tokens = 15,
    context_size = BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

Every effort moves you forward.

The first step is to understand the importance of your work


In [23]:
for param in model.parameters():
  param.requires_grad=False

In [24]:
model.out_head = torch.nn.Linear(BASE_CONFIG["emb_dim"], 30)

In [25]:
model

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768,

In [26]:
for param in model.trf_blocks[-1].parameters():
  param.requires_grad=True

for param in model.final_norm.parameters():
  param.requires_grad=True

In [27]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions", inputs.shape)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions torch.Size([1, 4])


In [28]:
with torch.no_grad():
  outputs = model(inputs)
print(outputs)

tensor([[[-1.3403e+00,  1.6740e+00, -2.1391e+00, -1.5196e+00, -2.2827e+00,
          -1.0842e+00,  1.0958e+00, -1.2350e+00,  1.4256e+00, -5.3516e-01,
           1.9268e+00, -2.0557e+00, -3.3562e+00,  2.4921e+00, -2.8471e+00,
          -5.0436e-01,  4.8762e-01, -3.8219e+00, -8.4622e-01,  1.5835e+00,
          -8.1541e-01,  1.0916e+00,  1.5297e-01,  9.6537e-01, -2.9998e+00,
          -1.9366e-01,  1.0422e+00,  1.8858e-01, -6.6791e-01, -2.2947e-01],
         [-7.4813e+00,  5.6588e+00, -9.0387e+00, -3.9087e+00, -1.2606e+01,
          -3.9699e+00,  2.9874e+00, -6.2640e+00,  7.4400e+00, -1.9896e+00,
           1.2141e+01, -3.5960e+00, -1.0531e+01,  5.4487e+00, -7.1550e+00,
           1.3336e+00,  3.4678e-01, -1.6510e+01, -5.8970e+00,  5.3332e+00,
          -3.3368e+00,  5.4462e+00, -1.8515e+00,  8.6037e+00, -1.3210e+01,
          -4.3487e+00,  3.6808e+00, -6.6142e-01, -4.5374e+00,  2.7694e+00],
         [-6.9446e+00,  5.6580e+00, -7.3395e+00, -3.0708e+00, -9.0102e+00,
          -4.2097e+00, 

In [29]:
logits = outputs[:,-1,:]

In [30]:
probas = torch.softmax(logits, dim=-1)

In [31]:
probas

tensor([[1.6561e-05, 9.4345e-02, 3.0107e-06, 6.1934e-05, 2.0852e-06, 1.5395e-05,
         1.7760e-02, 1.6966e-05, 1.0446e-02, 4.1479e-05, 4.5970e-01, 2.7926e-05,
         3.0164e-07, 2.6026e-01, 2.6372e-06, 2.0351e-03, 2.0995e-03, 5.5927e-08,
         6.2442e-05, 3.6876e-02, 7.0282e-04, 2.3704e-02, 1.6950e-04, 5.0138e-02,
         4.6355e-07, 9.0654e-04, 3.9023e-02, 9.8812e-04, 1.6955e-05, 5.8080e-04]])

In [32]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model = model.to(device)
    model.eval()
    correct_predictions, num_examples = 0, 0

    total_batches = len(data_loader) if num_batches is None else min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i >= total_batches:
            break
        # Move data to device
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)
        with torch.no_grad():
            outputs = model(input_batch)[:, -1, :]
        predicted_labels = torch.argmax(outputs, dim=-1)
        num_examples += predicted_labels.shape[0]
        correct_predictions += (predicted_labels == target_batch).sum().item()
    return correct_predictions / num_examples


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.manual_seed(123)

In [34]:
train_accuracy = calc_accuracy_loader(train_loader, model, device)

In [35]:
train_accuracy

0.012422360248447204

In [36]:
validation_accuracy = calc_accuracy_loader(validation_loader, model, device)
validation_accuracy

0.01358695652173913

In [37]:
def calc_loss_batch(input_batch, target_batch, model, device):
  input_batch, target_batch = input_batch.to(device), target_batch.to(device)
  logits = model(input_batch)[:,-1,:]
  loss = torch.nn.functional.cross_entropy(logits, target_batch)
  return loss

In [38]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
  total_loss = 0
  if len(data_loader) == 0:
    return float("nan")
  elif num_batches is None:
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))
  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      total_loss += loss.item()
    else:
      break
  return total_loss/num_batches

In [40]:
train_loss = calc_loss_loader(train_loader, model, device)
train_loss

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


**Fine tuning the model**

In [41]:
def train_classifier_simple(model, train_loader, validation_loader,
                            optimizer, device, num_epochs, eval_freq, eval_iter):
  train_losses, val_losses, train_accs, val_accs = [], [], [], []
  examples_seen, global_step = 0, -1

  for epoch in range(num_epochs):
    model.train()

    for input_batch, target_batch in train_loader:
      optimizer.zero_grad()
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      loss.backward()
      optimizer.step()
      examples_seen += input_batch.shape[0]
      global_step += 1

      if global_step % eval_freq == 0:
        train_loss, val_loss = evaluate_model(model, train_loader, validation_loader, device, eval_iter)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Ep {epoch+1} (Step {global_step:06d}): "
              f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
      train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
      train_accs.append(train_accuracy)
      val_accuracy = calc_accuracy_loader(validation_loader, model, device, num_batches=eval_iter)
      val_accs.append(val_accuracy)
      print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
      print(f"Validation accuracy: {val_accuracy*100:.2f}%")
  return train_losses, val_losses, train_accs, val_accs, examples_seen

In [42]:
def evaluate_model(model, train_loader, validation_loader, device, eval_iter):
  model.eval()
  with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
    val_loss = calc_loss_loader(validation_loader, model, device, num_batches=eval_iter)
  model.train()
  return train_loss, val_loss

In [43]:
import time
start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model = model,
    train_loader = train_loader,
    validation_loader = validation_loader,
    optimizer = optimizer,
    device = device,
    num_epochs = num_epochs,
    eval_freq = 50,
    eval_iter = 5
)

end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [43]:
device

device(type='cuda')

In [61]:
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()

    # Prepare inputs to the model
    input_ids = tokenizer.encode(text)
    supported_context_length = model.pos_emb.weight.shape[0]
    # Note: In the book, this was originally written as pos_emb.weight.shape[1] by mistake
    # It didn't break the code but would have caused unnecessary truncation (to 768 instead of 1024)

    # Truncate sequences if they too long
    input_ids = input_ids[:min(max_length, supported_context_length)]

    # Pad sequences to the longest sequence
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0) # add batch dimension

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the classified result
    return predicted_label

In [79]:
# Example call:
result = classify_review(
    text="floral dress for women",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=train_dataset.max_length,
    pad_token_id=50256
)
print(result)  # e.g., 'shoes_a12'


4


In [68]:
# Your category_map as provided
category_map = {
    'shoes_a12': 0,
    'makeup_a30': 1,
    'bottoms_a3': 2,
    'tops_a2': 3,
    'dresses_a1': 4,
    'underwear_lingerie_a7': 5,
    'fragrance_a31': 6,
    'skin_body_treatment_a32': 7,
    'outerwear_a11': 8,
    'sleepwear_a9': 9,
    'jewelry_a14': 10,
    'bags_a17': 11,
    'hosiery_a16': 12,
    'baby_accessories_a28': 13,
    'swimwear_a8': 14,
    'home_a35': 15,
    'hair_care_a33': 16,
    'eyewear_a19': 17,
    'hair_accessories_a20': 18,
    'headwear_a18': 19,
    'jumpsuits_coveralls_a4': 20,
    'personal_care_accessories_a34': 21,
    'belts__braces_a26': 22,
    'jacket_sportcoat_mg12': 23,
    'small_leather_goods_a23': 24,
    'food_a38': 25,
    'suits_sets_wardrobers_mg5': 26,
    'stationery_giftwrap_a36': 27,
    'shoe_care_a13': 28,
    'toys_games_a37': 29
}

# Invert the category_map to map indices back to category names
index_to_category = {v: k for k, v in category_map.items()}

# Example usage
text = "lipstick cetics"
predicted_label = classify_review(text, model, tokenizer, device, max_length=64, pad_token_id=50256)

# If condition to print the category name
if predicted_label in index_to_category:
    print(f"Predicted category: {index_to_category[predicted_label]}")
else:
    print(f"Unknown category index: {predicted_label}")


Predicted category: underwear_lingerie_a7
